# knowledge generation
논리지 생성 단계

# 1. 필요 라이브러리 설치
- Newspaper3k와 Selenium을 사용하여 웹 스크래핑을 수행합니다.
- Chromium-browser와 ChromeDriver는 Selenium이 웹 컨텐츠를 효과적으로 스크랩할 수 있도록 합니다.

In [176]:
!pip install --upgrade newspaper3k selenium
!pip install --upgrade chromedriver-autoinstaller
!sudo apt-get update
!sudo apt-get install chromium-browser

Hit:1 https://download.docker.com/linux/ubuntu jammy InRelease
Hit:2 https://dl.google.com/linux/chrome/deb stable InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]      
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease    
Fetched 229 kB in 6s (41.2 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-browser is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.


# 2. Python 스크립트
## 1) 라이브러리 불러오기
- 웹 스크래핑을 위한 필수 라이브러리를 불러옵니다.
- newspaper는 복잡한 웹페이지에서 콘텐츠를 추출하고, BeautifulSoup와 requests는 HTML 페이지를 파싱합니다.

In [177]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By


## 2) 뉴스 기사 URL 검색
네이버 뉴스를 통해 주어진 검색어로 최신 뉴스 기사 URL을 검색하고 추출합니다.

In [178]:
def get_news_urls(query):
    search_url = f"https://search.naver.com/search.naver?where=news&query={query}&sort=1"
    response = requests.get(search_url, headers={'User-agent': 'Mozilla/5.0'})
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        news_urls = [element['href'] for element in soup.find_all('a', class_='news_tit')]
        return news_urls
    else:
        print("Failed to fetch news URLs")
        return []


## 3)뉴스 기사 스크래핑
추출한 URL에서 뉴스 기사의 세부 정보를 추출합니다.

In [179]:
def scrape_news_articles(news_urls):
    articles_data = []
    for url in news_urls:
        try:
            article = Article(url)
            article.download()
            article.parse()
            articles_data.append({
                'url': url,
                'title': article.title,
                'authors': article.authors,
                'publish_date': article.publish_date,
                'text': article.text
            })
        except Exception as e:
            print(f"Failed to scrape article from {url}: {e}")
    return articles_data


## 4) 데이터 정리 및 Selenium 사용
- 서울 지하철 파업/연착/지연/사고/연장 검색
- DataFrame을 생성하고, 텍스트가 없거나 충분하지 않은 경우 Selenium으로 추가 데이터를 수집합니다.
- null 값이거나 텍스트 길이가 200자 이하 일때, 크롤링이 제대로 되지 않은 걸로 간주, selenium을 통해 크롤링
- '노컷뉴스' html 전처리

In [180]:
if __name__ == "__main__":
    queries = ["서울 지하철 파업", "서울 지하철 연착", "서울 지하철 지연","서울 지하철 사고", "서울 지하철 연장"]
    all_articles_data = []
    for query in queries:
        news_urls = get_news_urls(query)
        articles_data = scrape_news_articles(news_urls)
        for article_data in articles_data:
            article_data['category'] = query.split(" ")[-1]
        all_articles_data.extend(articles_data)
    df = pd.DataFrame(all_articles_data)
    df = df.replace('', np.nan)

    # Selenium 설정 및 데이터 추가 수집
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--remote-debugging-port=9222')  # This can help avoid some common errors
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    for index, row in df.iterrows():
        if pd.isna(row['text']) or len(row['text']) < 200:
            driver.get(row['url'])
            try:
                try:
                    fetched_text = driver.find_element(By.ID, 'pnlContent').text
                except:
                    fetched_text = driver.find_element(By.TAG_NAME, 'article').text
                df.at[index, 'text'] = fetched_text
            except Exception as e:
                print(f"Error fetching article from {row['url']}: {e}")
    driver.quit()

# 3. 데이터 확인

In [181]:
df.shape

(50, 6)

In [182]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   url           50 non-null     object
 1   title         50 non-null     object
 2   authors       50 non-null     object
 3   publish_date  44 non-null     object
 4   text          50 non-null     object
 5   category      50 non-null     object
dtypes: object(6)
memory usage: 2.5+ KB


In [183]:
df.head(50)

,url,title,authors,publish_date,text,category
0,http://www.labortoday.co.kr/news/articleView.h...,[백기완노나메기재단 신학철 이사장·최갑수 공동후원회장] “온몸으로 한국현대사 끌어안...,[],2024-05-27 07:30:04+09:00,"▲ 정기훈 기자\n\n“백기완이 깨부수려는 어둠의 계절은 끝났는가, 억울한 죽음은 ...",파업
1,https://news.heraldcorp.com/view.php?ud=202405...,"출산 장려금 지원 나선 MZ노조 위원장 ""저출산 피해, 20~40대 노동자가 가장 ...",[],2024-05-24 09:11:50+09:00,송시영 서울지하철공사 올바른노조위원장\n\n\n\n송시영 올바른노동조합 위원장 [연...,파업
2,https://www.yna.co.kr/view/AKR2024052306750000...,"서울교통공사 제3노조 ""타임오프 악용 노조간부 처벌·감사해야""",[],2024-05-23 11:36:04+09:00,"최윤선 기자\n\n'무단결근' 노조 간부 7명, 재심사 거쳐 전원 해임 처분\n\n...",파업
3,http://www.labortoday.co.kr/news/articleView.h...,근로시간면제 집단해고 신종 노조탄압인가,[],2024-05-21 07:30:03+09:00,권오훈 공인노무사(공공운수노조 부위원장)\n\n서울교통공사에서 노조 간부 36명에 ...,파업
4,https://www.dnews.co.kr/uhtml/view.jsp?idxno=2...,[데스크칼럼] 지하철 개통 50주년,[],None,"[데스크칼럼] 지하철 개통 50주년\n\n스무 살, 난생 처음 지하철이라는 걸 타 ...",파업
5,http://www.m-i.kr/news/articleView.html?idxno=...,[기획] “먹거리에 공공요금까지”… 물가 도미노 현실화,[],2024-05-02 12:00:00+09:00,누르던 먹거리 물가 터졌다… 외식업계 가격 줄인상\n\n정부 공공요금 동결…가스공사...,파업
6,http://www.m-i.kr/news/articleView.html?idxno=...,[기획] “물가 자극하는 대내외 악재 산적”…공공요금 하반기 줄인상 가능성↑,[],2024-05-02 14:39:44+09:00,물가 ‘복병’ 공공요금…도시가스·전기료 동결\n\n누적 적자 한계로 하반기 인상 가...,파업
7,https://biz.newdaily.co.kr/site/data/html/2024...,김밥부터 치킨·피자에 공공요금 인상까지… 가정의 달 물가 융단 폭격,[],2024-04-28 00:00:00,▲ 대형마트 전경. ⓒ뉴데일리DB\n\n김밥·치킨·커피 등 외식품목 가격과 가스·지...,파업
8,https://www.newsis.com/view/?id=NISX20240426_0...,가스·지하철 등 공공요금 도미노 인상 예고[끝모를 물가 충격②],[],2024-04-29 15:09:44+09:00,가스·전기료 5·6월 발표…재무 개선 압박\n3월 파업에 버스 요금 인상 가능성 커...,파업
9,http://weekly.chosun.com/news/articleView.html...,사모펀드에 포획된 버스 준공영제,[],2024-04-26 15:00:00+09:00,서울 시내버스가 12년 만에 총파업에 돌입한 지난 3월 28일 서울 소재 시내버스 ...,파업


# 4. 데이터 전처리
- publish_date의 null 값 처리를 위해 한번더 selenium 진행

In [184]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from dateutil import parser
import re

# Setup Chrome WebDriver with headless options and other configurations
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('blink-settings=imagesEnabled=false')  # 이미지 비활성화

# Setup service with ChromeDriverManager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.set_page_load_timeout(120)  # 타임아웃을 120초로 설정

# Assume 'df' is your DataFrame and it has columns 'url' and 'publish_date'
date_classes = ['news-date', 'date-repoter', 'article_info', 'article_byline', 'dates','dateFont','article_date']

for index, row in df.iterrows():
    if pd.isna(row['publish_date']):
        url = row['url']
        try:
            driver.get(url)
        except TimeoutException:
            print(f"Timeout while loading {url}")
            continue  # 다음 URL로 건너뛰기
        
        publish_date = None
        for date_class in date_classes:
            date_elements = driver.find_elements(By.CLASS_NAME, date_class)
            for date_element in date_elements:
                date_text = date_element.text
                numbers = re.findall(r'\d+', date_text)
                if len(numbers) >= 3:  # YYYY, MM, DD가 필요
                    publish_date = f"{numbers[0]}-{numbers[1].zfill(2)}-{numbers[2].zfill(2)}"
                    break
            if publish_date:
                break
        
        if publish_date:
            try:
                parsed_date = parser.parse(publish_date)
                df.at[index, 'publish_date'] = parsed_date
            except ValueError as e:
                print(f"Error parsing date from {url}: {e}")

# Close the browser
driver.quit()


Timeout while loading https://www.dnews.co.kr/uhtml/view.jsp?idxno=202405191523562810623
Timeout while loading https://news.kbs.co.kr/news/pc/view/view.do?ncd=7897246&ref=A
Timeout while loading http://www.kyeongin.com/main/view.php?key=20240522028366022
Timeout while loading https://www.kyeonggi.com/article/20240521580147
Timeout while loading https://www.ktv.go.kr/content/view?content_id=702259
Timeout while loading https://www.nocutnews.co.kr/news/6150378?utm_source=naver&utm_medium=article&utm_campaign=20240525100258


In [185]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# 셀레니움 웹 드라이버 설정
chrome_options = Options()
chrome_options.add_argument('--headless')  # 창 없는 모드
chrome_options.add_argument('--no-sandbox')  # 샌드박스 사용 안 함
chrome_options.add_argument('--disable-dev-shm-usage')  # /dev/shm 사용 안 함
chrome_options.add_argument('--disable-gpu')  # GPU 가속 사용 안 함
chrome_options.add_argument('blink-settings=imagesEnabled=false')  # 이미지 로드 안 함

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.set_page_load_timeout(30)  # 페이지 로딩 타임아웃 30초 설정

try:
    for index,row in df.iterrows():
        if row['title'] == 'KBS 뉴스':
            driver.get(row['url'])
            try:
                # 요소가 화면에 보일 때까지 기다린 후 해당 요소를 찾아 텍스트를 추출
                element = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.CSS_SELECTOR, "h4.headline-title"))
                    )
                real_title = element.text
                df.at[index, 'title'] = real_title  # 실제 제목으로 업데이트
            except (NoSuchElementException, TimeoutException, WebDriverException) as e:
                print(f"An error occurred for URL {row['url']}: {e}")
finally:
    driver.quit()  # 드라이버 종료


In [186]:
df.head(50)

,url,title,authors,publish_date,text,category
0,http://www.labortoday.co.kr/news/articleView.h...,[백기완노나메기재단 신학철 이사장·최갑수 공동후원회장] “온몸으로 한국현대사 끌어안...,[],2024-05-27 07:30:04+09:00,"▲ 정기훈 기자\n\n“백기완이 깨부수려는 어둠의 계절은 끝났는가, 억울한 죽음은 ...",파업
1,https://news.heraldcorp.com/view.php?ud=202405...,"출산 장려금 지원 나선 MZ노조 위원장 ""저출산 피해, 20~40대 노동자가 가장 ...",[],2024-05-24 09:11:50+09:00,송시영 서울지하철공사 올바른노조위원장\n\n\n\n송시영 올바른노동조합 위원장 [연...,파업
2,https://www.yna.co.kr/view/AKR2024052306750000...,"서울교통공사 제3노조 ""타임오프 악용 노조간부 처벌·감사해야""",[],2024-05-23 11:36:04+09:00,"최윤선 기자\n\n'무단결근' 노조 간부 7명, 재심사 거쳐 전원 해임 처분\n\n...",파업
3,http://www.labortoday.co.kr/news/articleView.h...,근로시간면제 집단해고 신종 노조탄압인가,[],2024-05-21 07:30:03+09:00,권오훈 공인노무사(공공운수노조 부위원장)\n\n서울교통공사에서 노조 간부 36명에 ...,파업
4,https://www.dnews.co.kr/uhtml/view.jsp?idxno=2...,[데스크칼럼] 지하철 개통 50주년,[],None,"[데스크칼럼] 지하철 개통 50주년\n\n스무 살, 난생 처음 지하철이라는 걸 타 ...",파업
5,http://www.m-i.kr/news/articleView.html?idxno=...,[기획] “먹거리에 공공요금까지”… 물가 도미노 현실화,[],2024-05-02 12:00:00+09:00,누르던 먹거리 물가 터졌다… 외식업계 가격 줄인상\n\n정부 공공요금 동결…가스공사...,파업
6,http://www.m-i.kr/news/articleView.html?idxno=...,[기획] “물가 자극하는 대내외 악재 산적”…공공요금 하반기 줄인상 가능성↑,[],2024-05-02 14:39:44+09:00,물가 ‘복병’ 공공요금…도시가스·전기료 동결\n\n누적 적자 한계로 하반기 인상 가...,파업
7,https://biz.newdaily.co.kr/site/data/html/2024...,김밥부터 치킨·피자에 공공요금 인상까지… 가정의 달 물가 융단 폭격,[],2024-04-28 00:00:00,▲ 대형마트 전경. ⓒ뉴데일리DB\n\n김밥·치킨·커피 등 외식품목 가격과 가스·지...,파업
8,https://www.newsis.com/view/?id=NISX20240426_0...,가스·지하철 등 공공요금 도미노 인상 예고[끝모를 물가 충격②],[],2024-04-29 15:09:44+09:00,가스·전기료 5·6월 발표…재무 개선 압박\n3월 파업에 버스 요금 인상 가능성 커...,파업
9,http://weekly.chosun.com/news/articleView.html...,사모펀드에 포획된 버스 준공영제,[],2024-04-26 15:00:00+09:00,서울 시내버스가 12년 만에 총파업에 돌입한 지난 3월 28일 서울 소재 시내버스 ...,파업


In [187]:
import spacy
import pandas as pd
import torch
from transformers import BertModel, BertTokenizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the Korean model from spaCy
nlp = spacy.load('ko_core_news_lg')

# Load KoBERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# Define related and action words
related_words = [
    "지하철", "교통", "운행", "노선", "승객", "출근", "전철", "메트로", "지하철역", "승강장", "지하철차량",
    "대중교통", "교통체계", "교통망", "환승", "전용차선"
]

action_words = [
    "파업", "지연", "연착", "중단", "정지", "혼잡", "운행지연", "운행중단", "서비스중단", "파업예고",
    "노동쟁의", "노조활동", "노사협상", "안전점검", "사고", "충돌", "부상", "사망", "운행변경",
    "노선변경", "시간표변경", "대체교통", "운행재개", "시위"
]

# Function to check dependency and entities
def check_dependency_and_entities(text):
    doc = nlp(text)
    for token in doc:
        if token.lemma_ in action_words:
            neighborhood = list(token.children) + list(token.ancestors) + list(token.subtree)
            for neighbor in neighborhood:
                if neighbor.lemma_ in related_words:
                    if any(child.dep_ == "neg" for child in neighbor.children):
                        continue
                    return True
                if neighbor.pos_ in ["ADP", "SCONJ"]:
                    connected_words = [child.lemma_ for child in neighbor.children]
                    if set(connected_words).intersection(related_words):
                        return True
    return False

# Function to get BERT embeddings
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# Function to check topic relevance using BERT embeddings
def check_topics(text, reference_texts):
    text_embedding = get_bert_embedding(text)
    reference_embeddings = [get_bert_embedding(ref) for ref in reference_texts]
    similarities = [cosine_similarity(text_embedding, ref_emb)[0][0] for ref_emb in reference_embeddings]
    return max(similarities) > 0.8  # Adjusted threshold for relevance

# Load your dataframe (assuming df is already defined and has a 'text' column)
reference_texts = [
    "지하철 파업으로 인한 운행 중단",
    "지하철 사고 발생",
    "지하철 연착 문제",
    "지하철 노선 변경",
    "지하철 운행 재개",
    "지하철 서비스 중단",
    "지하철 안전 점검",
    "지하철 충돌 사고",
    "지하철 승객 혼잡",
    "지하철 출근 시간 문제",
    "지하철 노조 파업",
    "지하철 운행 변경",
    "지하철 대체 교통"
] 

# Check if the document is relevant
def is_relevant_document(title):
    return check_dependency_and_entities(title) or check_topics(title, reference_texts)

# Assuming df is already defined and has a 'title' column
df['is_relevant'] = df['title'].apply(is_relevant_document)
df = df[df['is_relevant']]


In [ ]:
import getpass
import os

# 환경변수에 OpenAI API 키 저장 (사용자 입력으로 안전하게)
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [ ]:
!pip install langchain_openai


In [ ]:
pdf['title']

In [ ]:
pdf['indexed_title'] = pdf.index.astype(str) + ": " + pdf['title']


In [ ]:
titles = "\n".join(pdf['indexed_title'].tolist())
prompt = f"""
해당 기사 중에서 핵심 주제가 지하철 파업, 지연, 연착, 사고, 노선 연장인 기사들만 뽑아서 번호만 나열해줘:
{titles}
"""


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# prompt + model + output parser
llm = ChatOpenAI(model="gpt-4-turbo")
newsprompt = ChatPromptTemplate.from_template(prompt)
output_parser = StrOutputParser()

In [ ]:
newschain = newsprompt | llm | output_parser

In [ ]:
news_output1 = newschain.invoke({"titles": titles})

In [ ]:
news_output1